In [1]:
import sys
import numpy as np
sys.path.append('../')

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from mtcnn_torch import MTCNN_Torch
from faces_viewport import MTCNN_tf, ViewportsFaceDetector

C:\Users\paulo\anaconda3\envs\faces\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [2]:
dataset360 = pd.read_pickle('../dataframes/dataset360_detections.pkl')
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detectionsViewD_3,detectionsViewD_4,detectionsViewD_5,detectionsViewD_6,detectionsViewD_7,detectionsViewD_8,detectionsViewD_9,detectionsViewD_10,detections_sphere_mtcnn
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,35,-123,-1,"[[(1056, 367), (1083, 368), (1110, 366), (1053...","[[(1056, 367), (1083, 368), (1110, 366), (1114...","[[(1054, 375), (1081, 375), (1108, 374), (1108...","[[(1059, 375), (1085, 377), (1112, 377), (1111...","[[(1056, 373), (1083, 376), (1112, 378), (1111...","[[(1054, 377), (1080, 377), (1106, 376), (1110...","[[(1043, 380), (1070, 378), (1095, 376), (1106...","[[(1059, 375), (1085, 377), (1112, 377), (1111...","[[(1048, 381), (1072, 379), (1093, 376), (1105...","[[(1050, 380), (1076, 379), (1101, 376), (1108...",[]
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,85,-142,-1,"[[(1118, 537), (1174, 537), (1225, 530), (1100...","[[(1118, 537), (1174, 537), (1225, 530), (1269...","[[(1078, 552), (1143, 547), (1195, 535), (1245...","[[(1112, 546), (1162, 544), (1209, 538), (1249...","[[(1122, 544), (1174, 544), (1223, 539), (1260...","[[(1132, 540), (1189, 542), (1243, 537), (1275...","[[(1137, 545), (1191, 547), (1246, 544), (1276...","[[(1130, 554), (1195, 559), (1259, 555), (12, ...","[[(1099, 566), (1173, 566), (1241, 558), (3, 5...","[[(1143, 541), (1198, 546), (1256, 545), (1, 5...",[]
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-52,-81,-1,"[[(800, 120), (843, 110), (894, 104), (824, 15...","[[(800, 120), (843, 110), (894, 104), (901, 13...","[[(815, 119), (860, 117), (904, 120), (895, 14...","[[(803, 135), (834, 122), (876, 112), (892, 13...","[[(818, 134), (854, 129), (896, 127), (896, 14...","[[(803, 135), (834, 122), (876, 112), (892, 13...","[[(917, 91), (978, 95), (1031, 106), (1017, 12...","[[(811, 126), (855, 121), (901, 121), (897, 14...","[[(802, 128), (840, 118), (886, 112), (893, 14...","[[(818, 134), (854, 129), (896, 127), (896, 14...",[]
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-16,36,-1,"[[(462, 198), (509, 196), (556, 197), (466, 25...","[[(462, 198), (509, 196), (556, 197), (552, 25...","[[(456, 214), (495, 218), (530, 225), (530, 26...","[[(467, 212), (510, 213), (551, 218), (547, 25...","[[(460, 206), (506, 203), (554, 205), (550, 25...","[[(457, 211), (501, 206), (548, 205), (546, 25...","[[(467, 205), (511, 208), (551, 215), (547, 25...","[[(462, 216), (501, 209), (545, 206), (546, 25...","[[(457, 211), (498, 202), (547, 197), (549, 25...","[[(465, 209), (507, 207), (550, 209), (549, 25...",[]
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-71,140,-1,"[[(1100, 34), (1170, 2), (464, 31), (11, 61), ...","[[(1100, 34), (1170, 2), (464, 31), (264, 59),...","[[(1215, 51), (1264, 25), (326, 12), (244, 49)...","[[(1184, 40), (35, 15), (370, 25), (245, 57), ...","[[(1185, 31), (135, 14), (352, 32), (236, 62),...","[[(1174, 28), (170, 15), (346, 38), (234, 64),...","[[(1205, 44), (18, 19), (346, 19), (240, 55), ...","[[(1169, 23), (218, 17), (335, 43), (226, 67),...","[[(1166, 22), (227, 19), (329, 45), (222, 69),...","[[(1164, 19), (234, 19), (327, 46), (219, 69),...",erro


## MTCNN

In [3]:
mtcnn_torch = MTCNN_Torch()
#mtcnn_tf = MTCNN_tf(verbose = 1)

In [4]:
detections_mtcnn = dataset360.image_path.progress_apply(lambda x: mtcnn_torch.detect_faces_polys(x)[0])

In [7]:
dataset360['detections_mtcnn'] = detections_mtcnn

In [6]:
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detectionsViewD_3,detectionsViewD_4,detectionsViewD_5,detectionsViewD_6,detectionsViewD_7,detectionsViewD_8,detectionsViewD_9,detectionsViewD_10,detections_sphere_mtcnn,detections_mtcnn
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,35,-123,-1,"[[(1056, 367), (1083, 368), (1110, 366), (1053...","[[(1056, 367), (1083, 368), (1110, 366), (1114...","[[(1054, 375), (1081, 375), (1108, 374), (1108...","[[(1059, 375), (1085, 377), (1112, 377), (1111...","[[(1056, 373), (1083, 376), (1112, 378), (1111...","[[(1054, 377), (1080, 377), (1106, 376), (1110...","[[(1043, 380), (1070, 378), (1095, 376), (1106...","[[(1059, 375), (1085, 377), (1112, 377), (1111...","[[(1048, 381), (1072, 379), (1093, 376), (1105...","[[(1050, 380), (1076, 379), (1101, 376), (1108...",[],"[[(1054, 377), (1111, 377), (1111, 450), (1054..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,85,-142,-1,"[[(1118, 537), (1174, 537), (1225, 530), (1100...","[[(1118, 537), (1174, 537), (1225, 530), (1269...","[[(1078, 552), (1143, 547), (1195, 535), (1245...","[[(1112, 546), (1162, 544), (1209, 538), (1249...","[[(1122, 544), (1174, 544), (1223, 539), (1260...","[[(1132, 540), (1189, 542), (1243, 537), (1275...","[[(1137, 545), (1191, 547), (1246, 544), (1276...","[[(1130, 554), (1195, 559), (1259, 555), (12, ...","[[(1099, 566), (1173, 566), (1241, 558), (3, 5...","[[(1143, 541), (1198, 546), (1256, 545), (1, 5...",[],[]
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-52,-81,-1,"[[(800, 120), (843, 110), (894, 104), (824, 15...","[[(800, 120), (843, 110), (894, 104), (901, 13...","[[(815, 119), (860, 117), (904, 120), (895, 14...","[[(803, 135), (834, 122), (876, 112), (892, 13...","[[(818, 134), (854, 129), (896, 127), (896, 14...","[[(803, 135), (834, 122), (876, 112), (892, 13...","[[(917, 91), (978, 95), (1031, 106), (1017, 12...","[[(811, 126), (855, 121), (901, 121), (897, 14...","[[(802, 128), (840, 118), (886, 112), (893, 14...","[[(818, 134), (854, 129), (896, 127), (896, 14...",[],"[[(825, 116), (885, 116), (885, 176), (825, 17..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-16,36,-1,"[[(462, 198), (509, 196), (556, 197), (466, 25...","[[(462, 198), (509, 196), (556, 197), (552, 25...","[[(456, 214), (495, 218), (530, 225), (530, 26...","[[(467, 212), (510, 213), (551, 218), (547, 25...","[[(460, 206), (506, 203), (554, 205), (550, 25...","[[(457, 211), (501, 206), (548, 205), (546, 25...","[[(467, 205), (511, 208), (551, 215), (547, 25...","[[(462, 216), (501, 209), (545, 206), (546, 25...","[[(457, 211), (498, 202), (547, 197), (549, 25...","[[(465, 209), (507, 207), (550, 209), (549, 25...",[],"[[(466, 207), (548, 207), (548, 305), (466, 30..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-71,140,-1,"[[(1100, 34), (1170, 2), (464, 31), (11, 61), ...","[[(1100, 34), (1170, 2), (464, 31), (264, 59),...","[[(1215, 51), (1264, 25), (326, 12), (244, 49)...","[[(1184, 40), (35, 15), (370, 25), (245, 57), ...","[[(1185, 31), (135, 14), (352, 32), (236, 62),...","[[(1174, 28), (170, 15), (346, 38), (234, 64),...","[[(1205, 44), (18, 19), (346, 19), (240, 55), ...","[[(1169, 23), (218, 17), (335, 43), (226, 67),...","[[(1166, 22), (227, 19), (329, 45), (222, 69),...","[[(1164, 19), (234, 19), (327, 46), (219, 69),...",erro,[]


In [8]:
dataset360.to_pickle('../dataframes/dataset360_detections.pkl')

## Viewports MTCNN

In [12]:
dataset360 = pd.read_pickle('../dataframes/dataset360_detections.pkl')

In [16]:
#viewports_detector_tf = ViewportsFaceDetector(width = 200)

for density in range(3, 11):
    col_name = f'detectionsViewD_{density}'
    
    if col_name in dataset360.columns:
        print('density already computed')
        continue
    
    viewports_detector_torch = ViewportsFaceDetector(torch = True, width = 200, rows = density, cols = density*2)
    detections_viewport = dataset360.image_path.progress_apply(lambda x: viewports_detector_torch.detect_faces_polys(x)[0])
    dataset360[col_name] = detections_viewport
    dataset360.to_pickle('../dataframes/dataset360_detections.pkl')

In [17]:
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detectionsViewD_3,detectionsViewD_4,detectionsViewD_5,detectionsViewD_6,detectionsViewD_7,detectionsViewD_8,detectionsViewD_9,detectionsViewD_10
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,35,-123,-1,"[[(1056, 367), (1083, 368), (1110, 366), (1053...","[[(1056, 367), (1083, 368), (1110, 366), (1114...","[[(1054, 375), (1081, 375), (1108, 374), (1108...","[[(1059, 375), (1085, 377), (1112, 377), (1111...","[[(1056, 373), (1083, 376), (1112, 378), (1111...","[[(1054, 377), (1080, 377), (1106, 376), (1110...","[[(1043, 380), (1070, 378), (1095, 376), (1106...","[[(1059, 375), (1085, 377), (1112, 377), (1111...","[[(1048, 381), (1072, 379), (1093, 376), (1105...","[[(1050, 380), (1076, 379), (1101, 376), (1108..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,85,-142,-1,"[[(1118, 537), (1174, 537), (1225, 530), (1100...","[[(1118, 537), (1174, 537), (1225, 530), (1269...","[[(1078, 552), (1143, 547), (1195, 535), (1245...","[[(1112, 546), (1162, 544), (1209, 538), (1249...","[[(1122, 544), (1174, 544), (1223, 539), (1260...","[[(1132, 540), (1189, 542), (1243, 537), (1275...","[[(1137, 545), (1191, 547), (1246, 544), (1276...","[[(1130, 554), (1195, 559), (1259, 555), (12, ...","[[(1099, 566), (1173, 566), (1241, 558), (3, 5...","[[(1143, 541), (1198, 546), (1256, 545), (1, 5..."
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-52,-81,-1,"[[(800, 120), (843, 110), (894, 104), (824, 15...","[[(800, 120), (843, 110), (894, 104), (901, 13...","[[(815, 119), (860, 117), (904, 120), (895, 14...","[[(803, 135), (834, 122), (876, 112), (892, 13...","[[(818, 134), (854, 129), (896, 127), (896, 14...","[[(803, 135), (834, 122), (876, 112), (892, 13...","[[(917, 91), (978, 95), (1031, 106), (1017, 12...","[[(811, 126), (855, 121), (901, 121), (897, 14...","[[(802, 128), (840, 118), (886, 112), (893, 14...","[[(818, 134), (854, 129), (896, 127), (896, 14..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-16,36,-1,"[[(462, 198), (509, 196), (556, 197), (466, 25...","[[(462, 198), (509, 196), (556, 197), (552, 25...","[[(456, 214), (495, 218), (530, 225), (530, 26...","[[(467, 212), (510, 213), (551, 218), (547, 25...","[[(460, 206), (506, 203), (554, 205), (550, 25...","[[(457, 211), (501, 206), (548, 205), (546, 25...","[[(467, 205), (511, 208), (551, 215), (547, 25...","[[(462, 216), (501, 209), (545, 206), (546, 25...","[[(457, 211), (498, 202), (547, 197), (549, 25...","[[(465, 209), (507, 207), (550, 209), (549, 25..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-71,140,-1,"[[(1100, 34), (1170, 2), (464, 31), (11, 61), ...","[[(1100, 34), (1170, 2), (464, 31), (264, 59),...","[[(1215, 51), (1264, 25), (326, 12), (244, 49)...","[[(1184, 40), (35, 15), (370, 25), (245, 57), ...","[[(1185, 31), (135, 14), (352, 32), (236, 62),...","[[(1174, 28), (170, 15), (346, 38), (234, 64),...","[[(1205, 44), (18, 19), (346, 19), (240, 55), ...","[[(1169, 23), (218, 17), (335, 43), (226, 67),...","[[(1166, 22), (227, 19), (329, 45), (222, 69),...","[[(1164, 19), (234, 19), (327, 46), (219, 69),..."


## Sphere Net MTCNN

In [18]:
from sphere_mtcnn import SphereMTCNN_Torch

sphere_mtcnn = SphereMTCNN_Torch()

In [19]:

def sphere_aux(path):
    try:
        bounds = sphere_mtcnn.detect_faces_polys(path)[0]
    except:
        bounds = 'erro'
    return bounds
        

In [20]:
detections_sphere_mtcnn = dataset360.image_path.progress_apply(sphere_aux)

In [21]:
dataset360['detections_sphere_mtcnn'] = detections_sphere_mtcnn

In [22]:
dataset360.to_pickle('../dataframes/dataset360_detections.pkl')